In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
# Generate a sample pandas DataFrame
data = {'feature1': np.random.rand(100),
        'feature2': np.random.rand(100),
        'target': np.random.choice([1, 2], size=100)}

In [3]:
df = pd.DataFrame(data)

# Split the data into features and target
X = df[['feature1', 'feature2']].values
y = (df['target'] == 1).astype(int).values  # Convert to binary classification

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [6]:
# Define a simple neural network model
class BinaryClassificationModel(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassificationModel, self).__init__()
        self.fc = nn.Linear(input_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

In [7]:
# Instantiate the model and define loss function and optimizer
input_size = X_train.shape[1]
model = BinaryClassificationModel(input_size)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [8]:
# Training loop
epochs = 1000
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.view(-1, 1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [100/1000], Loss: 0.6903
Epoch [200/1000], Loss: 0.6899
Epoch [300/1000], Loss: 0.6895
Epoch [400/1000], Loss: 0.6892
Epoch [500/1000], Loss: 0.6888
Epoch [600/1000], Loss: 0.6885
Epoch [700/1000], Loss: 0.6882
Epoch [800/1000], Loss: 0.6879
Epoch [900/1000], Loss: 0.6876
Epoch [1000/1000], Loss: 0.6874


In [9]:
# Save the trained model
torch.save(model.state_dict(), 'binaryclassification.pt')
print("Model saved successfully!")

Model saved successfully!


In [10]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    predicted_labels = (predictions >= 0.5).float().view(-1).numpy()

In [11]:
accuracy = np.mean(predicted_labels == y_test)
print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.4500
